
このノートブックを実行するには、次の追加ライブラリが必要です。 Colab での実行は実験的なものであることに注意してください。問題がある場合は、Github の問題を報告してください。


In [ ]:
!pip install d2l==1.0.0-beta0



# コンパイラとインタプリタ

:label: `sec_hybridize`

これまで、この本では、 `print` 、 `+` 、 `if`などのステートメントを使用してプログラムの状態を変更する命令型プログラミングに焦点を当ててきました。次の単純な命令型プログラムの例を考えてみましょう。


In [1]:
def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g

print(fancy_func(1, 2, 3, 4))

10



Python は*インタプリタ型言語*です。上記の`fancy_func`関数を評価するとき、関数の本体を構成する操作が*順番に*実行されます。つまり、 `e = add(a, b)`評価し、結果を変数`e`として保存することで、プログラムの状態を変更します。次の 2 つのステートメント`f = add(c, d)`と`g = add(e, f)`も同様に実行され、加算が実行され、結果が変数として保存されます。 :numref: `fig_compute_graph`データの流れを示しています。 

![](http://d2l.ai/_images/computegraph.svg) :label: `fig_compute_graph`

命令型プログラミングは便利ですが、非効率的である可能性があります。一方で、 `add`関数が`fancy_func`を通じて繰り返し呼び出された場合でも、Python は 3 つの関数呼び出しを個別に実行します。これらが、たとえば GPU (または複数の GPU) で実行される場合、Python インタープリターから生じるオーバーヘッドが膨大になる可能性があります。さらに、 `fancy_func`内のすべてのステートメントが実行されるまで、 `e`と`f`の変数値を保存する必要があります。これは、ステートメント`e = add(a, b)`および`f = add(c, d)`実行後に変数`e`および`f`がプログラムの他の部分で使用されるかどうかがわからないためです。

## シンボリックプログラミング

代わりの*シンボリック プログラミング*を考えてみましょう。このプログラミングでは、通常、プロセスが完全に定義された後でのみ計算が実行されます。この戦略は、Theano や TensorFlow を含む複数の深層学習フレームワークで使用されています (後者は命令型拡張機能を取得しています)。通常、これには次の手順が含まれます。
1. 実行する操作を定義します。
1. 操作を実行可能プログラムにコンパイルします。
1. 必要な入力を入力し、コンパイルされたプログラムを呼び出して実行します。

これにより、大幅な最適化が可能になります。まず、多くの場合 Python インタープリターをスキップできるため、CPU 上の単一の Python スレッドと組み合わせられた複数の高速 GPU で重大になる可能性があるパフォーマンスのボトルネックが解消されます。次に、コンパイラは最適化して上記のコードを`print((1 + 2) + (3 + 4))`または`print(10)`に書き換える可能性があります。コンパイラはコードを機械語命令に変換する前に完全なコードを確認できるため、これが可能になります。たとえば、変数が不要になったときはいつでもメモリを解放できます (または割り当てをまったく行わないこともできます)。あるいは、コード全体を同等の部分に変換することもできます。より良いアイデアを得るために、以下の命令型プログラミング (結局のところ Python です) のシミュレーションを検討してください。


In [2]:
def add_():
    return '''
def add(a, b):
    return a + b
'''

def fancy_func_():
    return '''
def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
'''

def evoke_():
    return add_() + fancy_func_() + 'print(fancy_func(1, 2, 3, 4))'

prog = evoke_()
print(prog)
y = compile(prog, '', 'exec')
exec(y)


def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
print(fancy_func(1, 2, 3, 4))
10



命令型 (解釈型) プログラミングとシンボリック プログラミングの違いは次のとおりです。
- 命令型プログラミングの方が簡単です。 Python で命令型プログラミングを使用すると、コードの大部分は単純で簡単に作成できます。命令型プログラミング コードのデバッグも簡単になります。これは、関連するすべての中間変数値を取得して出力したり、Python の組み込みデバッグ ツールを使用したりする方が簡単だからです。
- シンボリック プログラミングはより効率的で移植が簡単です。シンボリック プログラミングを使用すると、コンパイル中のコードの最適化が容易になると同時に、プログラムを Python に依存しない形式に移植する機能もあります。これにより、プログラムを Python 以外の環境で実行できるようになり、Python インタープリターに関連する潜在的なパフォーマンスの問題が回避されます。

## ハイブリッドプログラミング

歴史的に、ほとんどの深層学習フレームワークは、命令型アプローチと記号的アプローチのどちらかを選択します。たとえば、Theano、TensorFlow (前者からインスピレーションを得た)、Keras、CNTK はモデルを記号的に定式化します。逆に、Chainer と PyTorch は命令型アプローチを採用しています。命令型モードは、後のリビジョンで TensorFlow 2.0 と Keras に追加されました。



上で述べたように、PyTorch は命令型プログラミングに基づいており、動的な計算グラフを使用します。シンボリック プログラミングの移植性と効率性を活用するために、開発者は両方のプログラミング モデルの利点を組み合わせることが可能かどうかを検討しました。これにより、ユーザーが純粋な命令型プログラミングを使用して開発およびデバッグできるようにすると同時に、ほとんどのプログラムを、製品レベルのコンピューティング パフォーマンスと展開が必要なときに実行できるシンボリック プログラムに変換する機能を備えた torchscript が誕生しました。



## `Sequential`クラスのハイブリッド化

ハイブリッド化がどのように機能するかを理解する最も簡単な方法は、複数の層を持つ深いネットワークを検討することです。従来、Python インタープリターはすべてのレイヤーのコードを実行して、CPU または GPU に転送できる命令を生成する必要がありました。単一の (高速な) コンピューティング デバイスの場合、これは大きな問題を引き起こしません。一方、AWS P3dn.24xlarge インスタンスなどの高度な 8 GPU サーバーを使用する場合、Python はすべての GPU をビジー状態に保つのに苦労します。ここではシングルスレッドの Python インタープリターがボトルネックになります。 `Sequential` `HybridSequential`に置き換えることによって、コードの重要な部分でこの問題にどのように対処できるかを見てみましょう。まず、単純な MLP を定義します。


In [3]:
import torch
from torch import nn
from d2l import torch as d2l


# Factory for networks
def get_net():
    net = nn.Sequential(nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 2))
    return net

x = torch.randn(size=(1, 512))
net = get_net()
net(x)

tensor([[ 0.1546, -0.0490]], grad_fn=<AddmmBackward0>)


`torch.jit.script`関数を使用してモデルを変換することで、MLP での計算をコンパイルして最適化することができます。モデルの計算結果は変わりません。


In [4]:
net = torch.jit.script(net)
net(x)

tensor([[ 0.1546, -0.0490]], grad_fn=<AddmmBackward0>)


これは真実であるにはあまりにうますぎるように思えます。前と同じコードを記述し、 `torch.jit.script`を使用してモデルを変換するだけです。これが起こると、ネットワークが最適化されます (以下でパフォーマンスのベンチマークを行います)。



### ハイブリッド化による高速化

コンパイルによって得られるパフォーマンスの向上を実証するために、ハイブリダイゼーションの前後で`net(x)`を評価するのに必要な時間を比較します。まずは今回測定するクラスを定義しましょう。これは、この章全体でパフォーマンスの測定 (および改善) を始める際に役立ちます。


In [5]:
#@save
class Benchmark:
    """For measuring running time."""
    def __init__(self, description='Done'):
        self.description = description

    def __enter__(self):
        self.timer = d2l.Timer()
        return self

    def __exit__(self, *args):
        print(f'{self.description}: {self.timer.stop():.4f} sec')


これで、torchscript を使用して 1 回、torchscript を使用せずに 1 回の計 2 回、ネットワークを呼び出すことができます。


In [6]:
net = get_net()
with Benchmark('Without torchscript'):
    for i in range(1000): net(x)

net = torch.jit.script(net)
with Benchmark('With torchscript'):
    for i in range(1000): net(x)

Without torchscript: 32.1398 sec


With torchscript: 30.5457 sec



上記の結果から分かるように、 `torch.jit.script`関数を使用して`nn.Sequential`インスタンスをスクリプト化した後、シンボリック プログラミングの使用によりコンピューティング パフォーマンスが向上します。



### 連載



モデルをコンパイルする利点の 1 つは、モデルとそのパラメーターをディスクにシリアル化 (保存) できることです。これにより、選択したフロントエンド言語に依存しない方法でモデルを保存できるようになります。これにより、トレーニングされたモデルを他のデバイスにデプロイし、他のフロントエンド プログラミング言語を簡単に使用できるようになります。同時に、多くの場合、コードは命令型プログラミングで達成できるものよりも高速になります。実際の`save`機能を見てみましょう。


In [7]:
net.save('my_mlp')
!ls -lh my_mlp*

-rw-rw-r-- 1 d2l-worker d2l-worker 652K Feb 10 04:38 my_mlp



## まとめ
- 命令型プログラミングでは、制御フローを使用してコードを記述でき、Python ソフトウェア エコシステムを大量に使用できるため、新しいモデルの設計が簡単になります。
- シンボリック プログラミングでは、プログラムを指定し、実行する前にコンパイルする必要があります。利点はパフォーマンスの向上です。



## 演習


1. 前の章で興味のあるモデルを確認してください。再実装することで計算パフォーマンスを改善できますか?



[ディスカッション](https://discuss.d2l.ai/t/2490)
